# Bee Field Collapse Simulation (Field Codex 3.0)

This notebook implements a simple **reaction–diffusion–interference** model for
bee population dynamics in a stylised Dutch landscape.

It corresponds to the equations in:
`docs/ecology/Bijensterfte-NL.md`

Main ideas:

- A bee population field `phi[x,y,t]` evolves over time.
- Habitat field `H[x,y]` determines carrying capacity and growth.
- Pesticide field `P[x,y]` and Varroa field `V[x,y]` cause mortality.
- Interference term models **synergy** between stressors.
- We track:
  - average bee density
  - habitat coherence C(t)
  - total interference load I(t)


In [0]:
import numpy as np
import matplotlib.pyplot as plt

# For reproducibility
rng = np.random.default_rng(42)

# Grid size (stylised landscape)
N = 80  # 80x80 grid

# Time steps
T = 200

# Model parameters
D = 0.15      # diffusion rate
alpha = 0.6   # growth rate
beta_P = 0.8  # pesticide mortality coefficient
beta_V = 0.6  # varroa mortality coefficient

# Interference coefficients (synergy)
gamma_PV = 0.9   # pesticide × varroa
gamma_PH = 0.5   # pesticide × habitat loss
gamma_VH = 0.4   # varroa × habitat loss

# Habitat coherence length for C(t)
xi = 5.0
H_threshold = 0.4


## Initialise fields

- `H`: habitat quality (0–1)
- `P`: pesticide pressure (0–1)
- `V`: varroa pressure (0–1)
- `phi`: bee population field


In [0]:
x = np.linspace(-1, 1, N)
X, Y = np.meshgrid(x, x)

# Habitat: several high-quality patches in an agricultural matrix
H = np.zeros((N, N))

# Three Gaussian habitat clusters
for (cx, cy, r) in [(-0.5, -0.4, 0.25), (0.4, 0.3, 0.3), (0.0, 0.7, 0.2)]:
    dist2 = (X - cx)**2 + (Y - cy)**2
    H += np.exp(-dist2 / (2 * r**2))

# Normalise to [0, 1]
H = H - H.min()
H = H / (H.max() + 1e-8)

# Pesticide field: strongest in agricultural belt
P = 0.6 * (1.0 - H)  # more pesticides where less habitat
P += 0.1 * rng.normal(size=(N, N))
P = np.clip(P, 0.0, 1.0)

# Varroa field: random clusters
V = np.zeros((N, N))
for (cx, cy, r) in [(0.2, -0.6, 0.25), (-0.3, 0.2, 0.2)]:
    dist2 = (X - cx)**2 + (Y - cy)**2
    V += np.exp(-dist2 / (2 * r**2))
V = V - V.min()
V = V / (V.max() + 1e-8)

# Initial bee population: small random fraction of carrying capacity
K = 1.0 + 2.0 * H   # carrying capacity rises with habitat quality
phi = 0.2 * K * (H > 0.3) * (1.0 + 0.2 * rng.normal(size=(N, N)))
phi = np.clip(phi, 0.0, None)


## Utility functions

### 1. Discrete Laplacian (diffusion)
Periodic boundary conditions for simplicity.

### 2. Habitat coherence \(C(t)\)

Simplified discrete version of the coherence definition:

\[
C = \frac{1}{|\mathcal{X}|}
\sum_{x} \frac{1}{Z} \sum_{y \in \mathcal{N}(x)}
\exp(-\|x-y\|/\xi) \; \mathbf{1}_{H(y) > h_0}
\]

We implement a local window with distance-based weights.


In [0]:
from scipy.ndimage import convolve

def laplacian(field):
    """Simple 2D Laplacian with periodic boundaries."""
    kernel = np.array([[0, 1, 0],
                       [1, -4, 1],
                       [0, 1, 0]])
    return convolve(field, kernel, mode="wrap")


def habitat_coherence(H, xi=5.0, threshold=0.4, window=7):
    """Approximate spatial coherence of habitat field.

    H: habitat field (N x N)
    xi: coherence length
    threshold: minimal habitat quality
    window: local neighbourhood size (odd integer)
    """
    N = H.shape[0]
    half = window // 2

    # Precompute distance-based weights
    coords = np.arange(-half, half+1)
    dX, dY = np.meshgrid(coords, coords)
    dist = np.sqrt(dX**2 + dY**2)
    weights = np.exp(-dist / max(xi, 1e-6))
    weights /= weights.sum()

    # Binary mask for H > threshold
    mask = (H > threshold).astype(float)

    # Local weighted fraction of good habitat around each cell
    local_good = convolve(mask, weights, mode="wrap")

    # Average over all cells
    C = local_good.mean()
    return C


## Simulation loop

We implement the discrete version of:

\[
\Phi_{t+1}(i) = \Phi_t(i)
+ D \Delta \Phi_t(i)
+ \alpha H(i)\Phi_t(i)(1 - \Phi_t(i)/K(i))
- \beta_P P(i)\Phi_t(i)
- \beta_V V(i)\Phi_t(i)
- I_{\text{tot}}(i)
\]

with

\[
I_{\text{tot}} = \gamma_{PV} P V \Phi
+ \gamma_{PH} P (1-H) \Phi
+ \gamma_{VH} V (1-H) \Phi
\]


In [0]:
phi_t = phi.copy()

mean_phi = []
coherence_series = []
interference_series = []

for t in range(T):
    # Diffusion term
    diff = D * laplacian(phi_t)

    # Growth term (logistic, modulated by habitat)
    growth = alpha * H * phi_t * (1.0 - phi_t / (K + 1e-8))

    # Mortality (pesticides + varroa)
    mort_P = beta_P * P * phi_t
    mort_V = beta_V * V * phi_t

    # Interference: synergy between stressors
    habitat_loss = (1.0 - H)
    I_tot = (
        gamma_PV * P * V * phi_t +
        gamma_PH * P * habitat_loss * phi_t +
        gamma_VH * V * habitat_loss * phi_t
    )

    # Update field (Euler step)
    phi_t = phi_t + diff + growth - mort_P - mort_V - I_tot
    phi_t = np.clip(phi_t, 0.0, None)

    # Record metrics
    mean_phi.append(phi_t.mean())
    coherence_series.append(habitat_coherence(H, xi=xi, threshold=H_threshold))
    interference_series.append(I_tot.mean())


## Visualisation

- Time series of mean bee density
- Interference load
- Habitat coherence (static, for this demo)
- Initial vs final bee field snapshots


In [0]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

ax1, ax2, ax3, ax4 = axes.ravel()

# 1. Mean bee density over time
ax1.plot(mean_phi)
ax1.set_title("Mean bee density")
ax1.set_xlabel("Time step")
ax1.set_ylabel("Mean Φ_bee")

# 2. Interference over time
ax2.plot(interference_series)
ax2.set_title("Average interference load")
ax2.set_xlabel("Time step")
ax2.set_ylabel("I_tot (mean)")

# 3. Habitat coherence (single value, but we plot as line for clarity)
ax3.plot(coherence_series)
ax3.set_title("Habitat coherence C(t)")
ax3.set_xlabel("Time step")
ax3.set_ylabel("C")

# 4. Final bee field
im = ax4.imshow(phi_t, origin="lower")
ax4.set_title("Final bee field Φ(x,y,T)")
plt.colorbar(im, ax=ax4, fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()


## Experimenting with scenarios

You can now:

- Reduce `P` (pesticides) and re-run the loop.
- Increase habitat quality `H` (e.g. smooth and raise values).
- Reduce `gamma_PV`, `gamma_PH`, `gamma_VH` to model lower interference.
- Increase diffusion `D` to simulate better connectivity.

You should see:

- In high-interference regimes → collapse of `mean_phi`.
- In restored regimes (higher H, lower P/V or gamma) → stabilisation or recovery.

This notebook is intentionally simple but expresses the **Field Codex 3.0** idea:

> We are not only tracking population size, but **field structure, coherence and interference**.